In [2]:
!pip install tensorflow-hub

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   --- ------------------------------------ 0.2/1.7 MB 5.0 MB/s eta 0:00:01
   --------- ------------------------------ 0.4/1.7 MB 4.2 MB/s eta 0:00:01
   ------------------- -------------------- 0.8/1.7 MB 6.5 MB/s eta 0:00:01
   --------------------- ------------------ 0.9/1.7 MB 5.4 MB/s eta 0:00:01
   -------------------------- ------------- 1.1/1.7 MB 4.8 MB/s eta 0:00:01
   --------------------------------- ------ 1.4/1.7 MB 5.1 MB/s eta 0:00:01
   ---------------------------------------  1.7/1.7 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 4.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import os
import logging

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppresses INFO and WARNING logs
tf.get_logger().setLevel(logging.ERROR)

# Load the Movenet model
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

# Edges for the connections between keypoints
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0, 255, 0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]

        if (c1 > confidence_threshold) & (c2 > confidence_threshold):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)

def overlay_image_on_region(frame, keypoints, image, region_keypoints):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    # Extract keypoints for the specified region
    keypoints_to_use = [shaped[kp] for kp in region_keypoints]

    # Check if any keypoint in the region has low confidence
    min_confidence = 0.5
    if any(kp[2] < min_confidence for kp in keypoints_to_use):
        return frame

    # Get the min and max coordinates for the bounding box
    min_x = min(kp[1] for kp in keypoints_to_use)
    max_x = max(kp[1] for kp in keypoints_to_use)
    min_y = min(kp[0] for kp in keypoints_to_use)
    max_y = max(kp[0] for kp in keypoints_to_use)

    # Calculate the width and height of the bounding box
    width = int(max_x - min_x)
    height = int(max_y - min_y)

    # Check if width and height are valid
    if width <= 0 or height <= 0:
        return frame

    # Resize the overlay image to fit the bounding box
    image_resized = cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

    # Calculate the top-left corner of the bounding box
    top_left_x = int(min_x)
    top_left_y = int(min_y)

    # Ensure the dimensions are within the frame
    bottom_right_x = min(top_left_x + image_resized.shape[1], x)
    bottom_right_y = min(top_left_y + image_resized.shape[0], y)

    # Check if the overlay is within the frame
    if bottom_right_x > x or bottom_right_y > y:
        return frame

    # Extract alpha channel from image if it exists
    if image_resized.shape[2] == 4:
        alpha_s = image_resized[:, :, 3] / 255.0  # Alpha channel
        alpha_f = 1.0 - alpha_s

        for c in range(0, 3):  # For RGB channels
            frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x, c] = (
                alpha_s * image_resized[:, :, c] + alpha_f * frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x, c]
            )
    else:
        # If no alpha channel, blend normally
        frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x] = cv2.addWeighted(
            frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x],
            0.5,
            image_resized[:bottom_right_y - top_left_y, :bottom_right_x - top_left_x],
            0.5,
            0
        )

    return frame

cap = cv2.VideoCapture(0)

# Load the images from the specified paths
muscles_img_path = 'C:/Users/irene/Downloads/muscleimage.jpeg'

# Load images
muscles_img = cv2.imread(muscles_img_path, cv2.IMREAD_UNCHANGED)

# Check if the images were loaded correctly
if muscles_img is None:
    print("Error: Could not load muscles image. Check the file path.")
    exit()
else:
    print("Images loaded successfully.")

# Create a window named 'Movement pose'
cv2.namedWindow('Movement pose', cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty('Movement pose', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 192, 256)
    input_img = tf.cast(img, dtype=tf.int32)

    # Detection
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:, :, :51].reshape((6, 17, 3))

    # Render keypoints and connections for all detected people
    for keypoints in keypoints_with_scores:
        draw_connections(frame, keypoints, EDGES, 0.10)
        draw_keypoints(frame, keypoints, 0.10)

        # Overlay muscular system image on torso up to neck
        frame = overlay_image_on_region(frame, keypoints, muscles_img, [5, 6, 15, 16])

    cv2.imshow('Movement pose', frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
